## Random  team generator

This uses the `w_mean` function from the elo calculation script to calculate team elos for every possible combination of players. Then, it calculates the standard deviation for each team's elos, multiplies this by a random value within the range of \\(\frac{[-\mathtt{random},\ \mathtt{random}]}{100}\\) and adds this onto the team elo. This randomness is meant to prevent unwanted repetition of teams.

In [11]:
from random import randrange
import numpy as np
import itertools


def w_mean(rankings, rankings_o):
    mean = sum(rankings_o) / len(rankings_o)
    diffs = []
    for r in rankings:
        diffs.append(abs(r - mean))
    weights = []
    for d in diffs:
        try:
            weights.append(d / sum(diffs))
        except ZeroDivisionError:
            weights.append(0)
    w_sum = sum(weights)
    if w_sum == 0:
        w_sum = len(rankings)
        weights = [1] * len(rankings)
    return sum([rankings[_] * weights[_] for _ in range(len(rankings))]) / sum(weights), weights


def combinations(players):
    teams = itertools.combinations(players, int(len(players) / 2))
    teams = list(teams)
    return [[teams[int(i - 1)], teams[-int(i)]] for i in range(1, int(len(teams) / 2) + 1, 1)]


def find_teams(players, mode, random=25, debug=False):
    all_combs = combinations(players)
    all_diffs = []
    for comb in all_combs:
        all_diffs.append(team_diff(comb[0], comb[1], mode, players, random))
    best_matchup = all_combs[np.argmin(all_diffs)]
    if debug: # I forgot to write a real debug
        print(all_diffs)
        return best_matchup, np.min(all_diffs)
    else:
        return best_matchup[::randrange(-1, 2, 2)] # randomize team 1 and team 2 selection


def team_elo(team, o_team, random):
    elo = w_mean(team, o_team)[0]
    std = (1 / (len(team) - 1) * sum([(x - elo) ** 2 for x in team])) ** .5
    return elo + std * randrange(-random, random) / 100 # naturally, sane values for random are [1, 100]


def team_diff(team1, team2, mode, players, random):
    R1, R2 = [], []

    if mode == "Escort":
        for i in range(len(team1)):
            R1.append(players[team1[i]]["emmr"])
            R2.append(players[team2[i]]["emmr"])
    elif mode == "Manhunt":
        for i in range(len(team1)):
            R1.append(players[team1[i]]["mhmmr"])
            R2.append(players[team2[i]]["mhmmr"])

    elo1 = team_elo(R1, R2, random)
    elo2 = team_elo(R2, R1, random)
    return abs(elo1 - elo2)


players = {"A" : {"mhmmr" : 1000}, "B" : {"mhmmr" : 1010}, "C" : {"mhmmr" : 980}, "D" : {"mhmmr" : 1050}, "E" : {"mhmmr" : 1200}, "F" : {"mhmmr" : 700}}

for x in range(10):
    r = randrange(1, 100)
    print("Randomness factor: ", r)
    print(find_teams(players, "Manhunt", r, False))


Randomness factor:  24
[('A', 'B', 'C'), ('D', 'E', 'F')]
Randomness factor:  30
[('A', 'E', 'F'), ('B', 'C', 'D')]
Randomness factor:  9
[('A', 'B', 'C'), ('D', 'E', 'F')]
Randomness factor:  17
[('A', 'B', 'C'), ('D', 'E', 'F')]
Randomness factor:  95
[('D', 'E', 'F'), ('A', 'B', 'C')]
Randomness factor:  72
[('A', 'E', 'F'), ('B', 'C', 'D')]
Randomness factor:  88
[('A', 'C', 'D'), ('B', 'E', 'F')]
Randomness factor:  8
[('A', 'B', 'C'), ('D', 'E', 'F')]
Randomness factor:  42
[('C', 'E', 'F'), ('A', 'B', 'D')]
Randomness factor:  89
[('C', 'E', 'F'), ('A', 'B', 'D')]
